In [1]:
from pytorch_pretrained_bert.modeling import BertPreTrainedModel, BertModel
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AutoTokenizer, AutoModel ,BertForTokenClassification,BertForSequenceClassification,TFBertForSequenceClassification,AdamW,BertTokenizer
from transformers import TransfoXLTokenizer, TransfoXLModel
from pytorch_pretrained_bert.modeling import BertModel
import torch
import logging
import torch
import time
import datetime
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
import gc
import random
import torch.nn as nn
import plotly.express as px
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
import nltk.data

tokenizer_nltk = nltk.data.load('tokenizers/punkt/english.pickle')
BATCH_SIZE=32
MAX_SEQ_LENGTH = 2400
tokenizer = AutoTokenizer.from_pretrained("berts/legal-bert-small-uncased") #dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
#model = BertModel.from_pretrained("berts/legal-bert-base-uncased")

c:\users\sichi\pycharmprojects\nlpproiect\venv\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\users\sichi\pycharmprojects\nlpproiect\venv\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
c:\users\sichi\pycharmprojects\nlpproiect\venv\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
c:\users\sichi\pycharmprojects\nlpproiect\venv\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger("regressor")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device="cpu"
n_gpu = torch.cuda.device_count()
logger.info("device: {} n_gpu: {}".format(device, n_gpu))

02/19/2021 22:27:48 - INFO - regressor -   device: cuda n_gpu: 1


In [3]:

df = pd.DataFrame()
df = pd.read_csv('input.csv', dtype={col:str for col in df.columns})
years = [str(i) for i in range(1960,2021)]

def clean_numbers(row,col):
    year = row['kpdate'].split('-')[0]
    article = row['appno']
    if article is None:
        article = row['extractedappno']
    for year in years:
        row[col] = row[col].replace(year,'YEAR')
    for x in article.split(';'):
        row[col] = row[col].replace(x,'APPNUMBER')
    if(re.search("^\s*[0-9]+\.",str(row['law']))):
        app_n = str(row['law']).split('.')[0] + '.'
        row[col] = row[col].replace(app_n,'NUMBER')
    return row[col]

def sliding_window(input_ids):
    #this bert only support a max length of 512 words
    to_return = []
    for slice_list in torch.split(input_ids,BATCH_SIZE,dim = 1):
        tens = model(slice_list)[0][:,0,:].detach().numpy()
        to_return.append(tens)
    print(len(to_return))
    return numpy.concatenate(to_return) #, dim=1

df['law'] = df['law'].apply(lambda x: str(x).lower())
for index, row in df.iterrows():
    df.at[index, 'law'] = clean_numbers(row,'law')
df['law'] = df['law'].apply(lambda x:tokenizer_nltk.tokenize(x))
df['law'] = df['law'].apply(lambda x:[x[0],x[1],x[2],x[len(x)-1]] if len(x) > 3 else x)

#df['law'] = df['law'].apply(lambda x: tokenizer.encode_plus(x,return_tensors = 'pt', padding=True,pad_to_multiple_of=BATCH_SIZE))
#df['law'] = df['law'].apply(lambda x: tokenizer.tokenize(x))
df['kpdate'] = df['kpdate'].apply(lambda date: float("{:.2f}".format(int(date.split('-')[0]) + float((int(date.split('-')[1]) - 1)/12))))
minim_date = min(df['kpdate'])
df['kpdate'] = df['kpdate'].apply(lambda date: date-minim_date)

In [4]:
min([len(x) for x in df['law']])

1

In [5]:
def format_time(elapsed):
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [6]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = [np.argmax(x) for x in preds]
  #  pred_flat = np.argmax(preds, axis=1).flatten()
#     labels_flat = labels#.flatten()
#     return np.sum(pred_flat == labels_flat) / len(labels_flat)
    return mean_absolute_error(pred_flat,labels)

In [7]:
def children(m):
    return m if isinstance(m, (list, tuple)) else list(m.children())


def set_trainable_attr(m, b):
    m.trainable = b
    for p in m.parameters():
        p.requires_grad = b


def apply_leaf(m, f):
    c = children(m)
    if isinstance(m, nn.Module):
        f(m)
    if len(c) > 0:
        for l in c:
            apply_leaf(l, f)
def count_model_parameters(model):
    logger.info(
        "# of paramters: {:,d}".format(
            sum(p.numel() for p in model.parameters())))
    logger.info(
        "# of trainable paramters: {:,d}".format(
            sum(p.numel() for p in model.parameters() if p.requires_grad)))

In [8]:
# attention_masks = []
# for sent in input_ids:
    
#     att_mask = [int(float(token_id) > 0) for token_id in sent]
#     attention_masks.append(att_mask)


train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(df['law'], df['kpdate'].values, 
                                                            random_state=2018, test_size=0.1)
# train_masks, validation_masks, _, _ = train_test_split(attention_masks, df['kpdate'].values,
#                                              random_state=2018,test_size=0.1)

In [9]:
# import nltk.data

# tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# train_inputs = train_inputs.apply(lambda x:tokenizer.tokenize(x))
# validation_inputs = validation_inputs.apply(lambda x:tokenizer.tokenize(x))

In [10]:
df['law'] = df['law'].apply(lambda x:tokenizer_nltk.tokenize(x))
maxims = np.array([max([len(y) for y in d]) for d in df['law']]) #7121
plt.hist(maxims);
plt.show()
print(len(df['law'][0]))

TypeError: expected string or bytes-like object

In [ ]:
train_inputs_ids = []
train_attention_masks = []

train_labels_temp = []
for i,train_input in enumerate(train_inputs):
    
    for j in range(len(train_input)):
        train_labels_temp.append(train_labels[i])
        
    for sent in train_input:
        encoded_dict = tokenizer.encode_plus(
                            sent,
                            padding='max_length',
                            truncation=True,
                            add_special_tokens = True,
                            max_length = 512,          # Truncate all sentences.
                            return_tensors = 'pt',     # Return pytorch tensors.
                       )
        train_inputs_ids.append(encoded_dict['input_ids'])
        train_attention_masks.append(encoded_dict['attention_mask'])

train_labels = train_labels_temp
        
validation_inputs_ids = []
validation_attention_masks = []
validation_labels_temp = []
validation_length = []
for i,validation_input in enumerate(validation_inputs):
    
    validation_length.append(len(validation_input))
    for j in range(len(validation_input)):
        validation_labels_temp.append(validation_labels[i])
 
    for sent in validation_input:
        encoded_dict = tokenizer.encode_plus(
                            sent,
                            padding='max_length',
                            truncation=True,
                            add_special_tokens = True,
                            max_length = 512,          # Truncate all sentences.
                            return_tensors = 'pt',     # Return pytorch tensors.
                       )
        validation_inputs_ids.append(encoded_dict['input_ids'])
        validation_attention_masks.append(encoded_dict['attention_mask'])

validation_labels = validation_labels_temp

In [ ]:
train_inputs.values[0][0]

In [ ]:
train_inputs_ids = torch.cat(train_inputs_ids,dim=0)
train_attention_masks = torch.cat(train_attention_masks,dim=0)
validation_inputs_ids = torch.cat(validation_inputs_ids,dim=0)
validation_attention_masks = torch.cat(validation_attention_masks,dim=0)

In [ ]:
train_labels = torch.tensor(train_labels, dtype=torch.float)
validation_labels = torch.tensor(validation_labels, dtype=torch.float)

In [ ]:
batch_size = 1
train_data = TensorDataset(train_inputs_ids,train_attention_masks,train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs_ids,validation_attention_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [ ]:
#class BertForSequenceRegression(BertModel):
class BertForSequenceRegression(BertModel):
    def __init__(self, config):
        super(BertForSequenceRegression, self).__init__(config)
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.regressor = nn.Linear(config.hidden_size, 1)
        self.apply(self.init_bert_weights)
        self.loss_fct = torch.nn.MSELoss()

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, targets=None):
        _, pooled_output = self.bert(input_ids, token_type_ids, attention_mask, output_all_encoded_layers=False)
        pooled_output = self.dropout(pooled_output)
        outputs = self.regressor(pooled_output).clamp(-1, 1)
        if targets is not None:
            loss = self.loss_fct(outputs.view(-1), targets.view(-1))
            return loss
        else:
            return outputs

In [ ]:
def set_trainable(l, b):
    apply_leaf(l, lambda m: set_trainable_attr(m, b))

model = BertForSequenceRegression.from_pretrained("berts/legal-bert-base-uncased")
set_trainable(model, True)
# Tell pytorch to run this model on the GPU.
model.cuda()

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8
                )
from transformers import get_linear_schedule_with_warmup
# Number of training epochs (authors recommend between 2 and 4)
epochs = 4
# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs
# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [ ]:
gc.collect()

In [ ]:
# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128
# Set the seed value all over the place to make this reproducible.
# seed_val = 42
# random.seed(seed_val)
# np.random.seed(seed_val)
# torch.manual_seed(seed_val)
# torch.cuda.manual_seed_all(seed_val)
# Store the average loss after each epoch so we can plot them.
loss_values = []
# For each epoch...
epochs = 4
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    # Measure how long the training epoch takes.
    t0 = time.time()
    # Reset the total loss for this epoch.
    total_loss = 0
    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()
    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):
        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        
        # Perform a forward pass (evaluate the model on this training batch).
        # This will return the loss (rather than the model output) because we
        # have provided the `labels`.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    targets=b_labels)
        
        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple.
        loss = outputs
        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_loss += loss.item()
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()
        # Update the learning rate.
        scheduler.step()
    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.
    print("")
    print("Running Validation...")
    t0 = time.time()
    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()
    # Tracking variables
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    y_pred = np.array([])
    y_true = np.array([])
    total_eval_loss = []
    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        
            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        
        # Accumulate the validation loss.
        eval_loss += loss.item()
        
        logits = outputs.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        y_pred = np.append(y_pred,logits)
        y_true = np.append(y_true,label_ids)
        
        validation_time = format_time(time.time() - t0)
        
        print("  Validation took: {:}".format(validation_time))
        
        # Calculate the accuracy for this batch of test sentences.
        #tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        # Accumulate the total accuracy.
        nb_eval_steps += 1
    # Report the final accuracy for this validation run.

    # Calculate the average loss over all of the batches.
    avg_val_loss = eval_loss / len(validation_dataloader)
    total_eval_loss.append(avg_val_loss)
    # Measure how long the validation run took.
    
    validation_time = format_time(time.time() - t0)
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Mae: {0:.2f}".format(mean_absolute_error(y_true,y_pred)))
    print("  Validation took: {:}".format(validation_time))
print("")
print("Training complete!")

In [ ]:
f = pd.DataFrame(loss_values)
f.columns=['Loss']
fig = px.line(f, x=f.index, y=f.Loss)
fig.update_layout(title='Training loss of the Model',
                   xaxis_title='Epoch',
                   yaxis_title='Loss')
fig.show()

In [ ]:
# Prediction on test set
#print('Predicting labels for {:,} test sentences...'.format(len(prediction_inputs)))
# Put model in evaluation mode
model.eval()
# Tracking variables 
predictions , true_labels = [], []
# Predict 
dataloader_iterator = iter(validation_dataloader)
for l in validation_length:
    logits_sum = 0
    labels_sum = 0
    for i in range(l):
        batch = next(iter(validation_dataloader))
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)

        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch

        # Telling the model not to compute or store gradients, saving memory and 
        # speeding up prediction
        with torch.no_grad():
          # Forward pass, calculate logit predictions
          outputs = model(b_input_ids, token_type_ids=None, 
                          attention_mask=b_input_mask)
        logits = outputs
        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        logits_sum = logits_sum + logits[0]
        labels_sum = labels_sum + label_ids[0]
    # Store predictions and true labels
    predictions.append(float(logits/l))
    true_labels.append(float(label_ids/l))

print(flat_accuracy(predictions,true_labels)) #11.90282330480998
print('DONE.')

In [ ]:
print(mean_absolute_error(logits,label_ids))
print('DONE.')

In [ ]:
# maxim = 0
# for i in range(0,len(X_combined)):
#     #print(len(X_combined[i].input_ids),len(X_combined[i].input_mask),len(X_combined[i].segment_ids),X_combined[i].target)
#     if len(X_combined[i].input_ids) > maxim:
#         maxim = len(X_combined[i].input_ids)
# print(maxim) #179094

In [ ]:
for i in range(0,len(predictions)):
    predictions[i] = np.argmax(predictions[i], axis=1).flatten()
print predictions

In [ ]:
for i in range(0,len(predictions)):
    predictions[i] = np.argmax(predictions[i], axis=1).flatten()
print predictions

In [ ]:
# maxim = 0
# for i in range(0,len(X_combined)):
#     #print(len(X_combined[i].input_ids),len(X_combined[i].input_mask),len(X_combined[i].segment_ids),X_combined[i].target)
#     if len(X_combined[i].input_ids) > maxim:
#         maxim = len(X_combined[i].input_ids)
# print(maxim) #179094

In [ ]:
for i in range(0,len(predictions)):
    predictions[i] = np.argmax(predictions[i], axis=1).flatten()
print predictions

In [ ]:
# maxim = 0
# for i in range(0,len(X_combined)):
#     #print(len(X_combined[i].input_ids),len(X_combined[i].input_mask),len(X_combined[i].segment_ids),X_combined[i].target)
#     if len(X_combined[i].input_ids) > maxim:
#         maxim = len(X_combined[i].input_ids)
# print(maxim) #179094

In [ ]:
# maxim = 0
# for i in range(0,len(X_combined)):
#     #print(len(X_combined[i].input_ids),len(X_combined[i].input_mask),len(X_combined[i].segment_ids),X_combined[i].target)
#     if len(X_combined[i].input_ids) > maxim:
#         maxim = len(X_combined[i].input_ids)
# print(maxim) #179094

In [ ]:
# from fastprogress import master_bar, progress_bar
# import gc
# import pickle

# def set_trainable(l, b):
#     apply_leaf(l, lambda m: set_trainable_attr(m, b))

# #del X_train
# gc.collect()
# model = BertForSequenceRegression.from_pretrained("berts/legal-bert-base-uncased")
# #train all layers
# set_trainable(model, True)
# #set_trainable(model.embeddings, False)
# #set_trainable(model.encoder, False)
# count_model_parameters(model)
# # with open('train_data','rb') as f:
# #     train_data = pickle.load(f)
# #train_sampler = RandomSampler(train_dataloader)
# #train_dataloader = DataLoader(train_dataloader, sampler=train_sampler, batch_size=32)
# train(model,train_dataloader, num_epochs = 1,train_features_lenth = len(train_dataloader))


In [ ]:
# train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)
# from fastprogress import master_bar, progress_bar
# import gc
# import pickle

# def set_trainable(l, b):
#     apply_leaf(l, lambda m: set_trainable_attr(m, b))

# #del X_train
# gc.collect()
# model = BertForSequenceRegression.from_pretrained("berts/legal-bert-base-uncased")
# model.eval()
# model.to(device)
# #train all layers
# set_trainable(model, True)
# #set_trainable(model.embeddings, False)
# #set_trainable(model.encoder, False)
# count_model_parameters(model)
# # with open('train_data','rb') as f:
# #     train_data = pickle.load(f)
# #train_sampler = RandomSampler(train_dataloader)
# #train_dataloader = DataLoader(train_dataloader, sampler=train_sampler, batch_size=32)
# train(model,train_dataloader, num_epochs = 1,train_features_lenth = len(train_dataloader))


In [ ]:
# # Save a trained model
# model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self
# output_model_file = "./regressor_legalbert.pth"
# torch.save(model_to_save.state_dict(), output_model_file)

In [ ]:
# # Load a trained model that you have fine-tuned
# model_state_dict = torch.load(output_model_file)
# model = BertForSequenceRegression.from_pretrained("berts/legal-bert-base-uncased", state_dict=model_state_dict)
# model.to(device)

In [ ]:
# y_combined = convert_examples_to_features(y_combined, MAX_SEQ_LENGTH, tokenizer)

# all_input_ids = torch.tensor([f.input_ids for f in y_combined], dtype=torch.long)
# all_input_mask = torch.tensor([f.input_mask for f in y_combined], dtype=torch.long)
# all_segment_ids = torch.tensor([f.segment_ids for f in y_combined], dtype=torch.long)
# all_targets = torch.tensor([f.target for f in y_combined], dtype=torch.float)
# print(all_input_ids.shape,all_input_mask.shape,all_segment_ids.shape,all_targets.shape)

# eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_targets)
# eval_sampler = RandomSampler(train_data)
# eval_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

In [ ]:
# import pickle
# with open('eval_data', 'wb') as f:
#     pickle.dump(eval_data,f)

# with open('train_data', 'wb') as f:
#     pickle.dump(train_data,f)


In [ ]:
# from fastprogress import progress_bar,master_bar

# model.eval()
# eval_loss, eval_accuracy = 0, 0
# nb_eval_steps, nb_eval_examples = 0, 0

# outputs = []
# mb = progress_bar(eval_dataloader)
# for input_ids, input_mask, segment_ids, targets in mb:
#     input_ids = input_ids.to(device)
#     input_mask = input_mask.to(device)
#     segment_ids = segment_ids.to(device)
#     targets = targets.to(device)

#     with torch.no_grad():
#         #tmp_eval_loss = model(input_ids, segment_ids, input_mask, targets)
#         outputs.append(model(input_ids, segment_ids, input_mask))

#     # outputs = outputs.detach().cpu().numpy()
#     # targets = targets.to('cpu').numpy()
#     # tmp_eval_accuracy = accuracy(logits, label_ids)

#     #eval_loss += tmp_eval_loss.mean().item()
#     # eval_accuracy += tmp_eval_accuracy

#     nb_eval_examples += input_ids.size(0)
#     nb_eval_steps += 1
#     mb.comment = f'{eval_loss / nb_eval_steps:.4f}'

# print(eval_loss / nb_eval_steps)

In [ ]:
# for i in range(0,len(outputs)):
#     print(outputs[i].argmax(0))